# 5. 스트림릿(Streamlit)으로 챗봇 구현하기
## 5.1 스트림릿 기본 사용 방법
### 5.1.1. 스트림릿 설치

In [ ]:
pip install streamlit

- 에러가 발생한다면..(no module named 'streamlit.cli')
- streamlit-script.py 파일을 찾아서
- from streamlit.web.cli import main    => .web 추가해주기

### 5.1.2 기본 사용방법

In [ ]:
# st_test1.py 파일로 만들어 실행
# 실행은 python -m streamlit run ./gemini_api/streamlit/st_test1.py
import streamlit as st
text = "마지막 레이어의 로짓값을 가정"
st.header(text, divider='rainbow')
st.subheader(text)
st.title(text)
st.write(text)
st.write("# Bar Chart")
vocab_logits = {"나는": 0.01,"내일": 0.03,"오늘": 0.25,"어제": 0.3,
                "산에": 0.4,"학교에": 0.5,"집에": 0.65,
                "오른다": 1.2,"간다": 1.05,"왔다": 0.95}
st.bar_chart(vocab_logits)
st.caption(text)

- ![스트림릿공식문서](https://docs.streamlit.io/develop/api-reference)

## 5.2. 스트림릿 핵심 콘셉트
### 5.2.1. 반응형 프로그래밍
![스트림릿동작원리](https://wikidocs.net/images/page/231440/%ED%95%B5%EC%8B%AC%EC%BD%98%EC%85%89%ED%8A%B8.png)
- 스트림릿 환경에서 수행되는 파이썬 스크립트는 스트림리셍서 관리하는 프론트엔드 엔진으로 전달, 화면상의 컴포넌트로 출력
- 스트림릿의 프론트엔드 엔진은 SPA(Single Page Application)로 동작. 
    - 최초 출력 이후에는 웹 페이지 전체를 리로드하지 않음.
    - 서버로부터 받은 데이터나 정보를 바탕으로 화면 업데이트
    - 화면상의 컴포넌트에 인터렉션을 가하면, 스트림릿 프레임워크는 애플리케이션의 상태를 최신화하기 위해 파이썬 스크립트 전체를 다시 실행함.

### 5.2.2. 데이터 캐싱

- 전체 스크립트를 다시 실행하면 개발 과정은 단순화 됨.
    - 하지만, 중복된 내용을 재실행하는 문제점이 존재
- 파이썬 스크립 전체가 다시 실행될 때의 대기시간 발생 문제를 개선하기 위해 캐싱 매커니즘 제공
    - @st.cache_data 데코레이터 사용하면 함수 실행 결과가 캐시에 보관됨.
    - 이후 파라미터가 달라졌거나 구현 내용이 달라진 경우에 한해서만 함수를 다시 실행.
    - 함수 코드가 변경되지 않았고, 이전에 호출된 파라미터와 동일하다면, 캐시에 보관되어 있는 결과를 반환함으로써 불필요한 재실행을 막음.

In [ ]:
# 활용 예시
import streamlit as st
import time

@st.cache_data     # 해당 부분이 캐시 사용
def get_vocab_logits(param=0):
    print(f"get_vocab_logits({param}) starting")
    time.sleep(10)
    vocab_logits = {"나는": 0.01,"내일": 0.03,"오늘": 0.25,"어제": 0.3,
                    "산에": 0.4,"학교에": 0.5,"집에": 0.65,
                    "오른다": 1.2,"간다": 1.05,"왔다": 0.95}
    vocab_logits = {word: logit + param for word, logit in vocab_logits.items()}
    print(f"get_vocab_logits({param}) ending")    
    return vocab_logits

text = "마지막 레이어의 로짓값을 가정"
st.header(text, divider='rainbow')
st.subheader(text)
st.title(text)
st.write(text)

user_input = st.number_input(label="로짓값에 더해지는 숫자를 입력하세요.", value=0)

st.write("# Bar Chart")
st.bar_chart(get_vocab_logits(user_input))
st.caption(text)

- 주의사항
    - @st.cache_data는 함수에 전달되는 파라미터와 구현 내용이 동일하다면 함수의 호출 결과 역시 항상 동일할거라는 가정 하에 캐싱 메커니즘을 적용
    - 함수 내에서 데이터베이스에 접근하거나 네트워크 통신 등을 통해 그 결과가 변경되도록 코드가 구현되어 있다면 동작하지 않을 수 있음.
    - 함수 내의 전역변수 같은 외부 요소에 의존하는 경우에도 올바르게 반영되지 않을 수 있음.

### 5.3.3. 리소스 캐싱

- @st.cache_resource 데코레이터
    - 머신러닝 모델이나 데이터베이스 커네션 등의 리소스를 효율적으로 사용하도록 도움.
    - 프로그램에 사용되는 리소스를 한 번만 로드하거나 연결하고, 이후의 호출에서는 캐싱된 인스턴스를 재사용함으로써 처리 시간을 단축시킴
- @st.cache_data를 사용하면 함수에서 산출된 최종 결괏값을 복제하여 반환함.
- @st.cache_resource를 사용하면 함수의 결괏값을 가리키는 참조(레퍼런스)를 반환함.<br>
![](https://wikidocs.net/images/page/231796/st_cache_data%EB%A5%BC_%EC%82%AC%EC%9A%A9%ED%95%98%EB%8A%94%EA%B2%BD%EC%9A%B0.png) ![](https://wikidocs.net/images/page/231796/st_cache_resource%EB%A5%BC_%EC%82%AC%EC%9A%A9%ED%95%98%EB%8A%94%EA%B2%BD%EC%9A%B0.png)

- @st.cache_data
    - 직렬화가 가능하고 데이터의 불변성이 필요한 경우 사용
    - 복잡한 연산 결과를 재사용하려고 할 때
- @st.cache_resource
    - 직렬화할 수 없으며 결과를 전역적으로 공유해야 하는 경우
    - 프로그램 내에서 자원을 효율적으로 관리하고 싶을 때

### 5.2.4 세션 상태 관리

- 사용자와 상호작용하는 챗봇을 구현하려면 사용자별 세션 정보가 필요
- 만약 멀티턴 대화를 한다면 챗봇과 사용자가 나눈 대화 이력을 모델의 입력값으로 전달해야 하므로 동시에 여러 명의 사용자가 접속한다면 사용자별로 서로 다른 이력이 관리되어야 함.
- 스트림릿 프레임워크는 session_state 객체를 통해 세션 정보를 관리할 수 있게 한다.

## 5.3. 제미나이 챗봇 만들기

### 5.3.1. 메시지 컨테이터

- st.chat_input 메서드를 호출하면 입력메시지 컨테이너가 나타남.

    - `prompt = st.chat_input("메시지를 입력하세요.")`
    

```
import streamlit as st
prompt = st.chat_input("메시지를 입력하세요.")
if prompt:
    with st.chat_message("user"):
        st.write(prompt)
    with st.chat_message("ai", avatar="🤖"):
        st.write("이것은 인공지능 응답입니다.")
```

- st.chat_message(name="이름", avatar="이모지") 형태로 작성가능
    - user, assistant, ai, human 의 경우 사전 정의된 아바타가 출력


In [ ]:
import streamlit as st
st.title("Auto_Essay_Scoring System")

if "chat_history" not in st.session_state:
    st.session_state.chat_history = []

for content in st.session_state.chat_history:
    with st.chat_message(content["role"]):
        st.markdown(content['message'])    

if prompt := st.chat_input("메시지를 입력하세요."):
    with st.chat_message("user"):
        st.markdown(prompt)
        st.session_state.chat_history.append({"role": "user", "message": prompt})

    with st.chat_message("ai"):                
        response = f'{prompt}... {prompt}... {prompt}...'
        st.markdown(response)
        st.session_state.chat_history.append({"role": "ai", "message": response})

### 5.3.2. 제미나이 챗봇 단계별 구현하기

In [ ]:
# st_chatbot.py 파일의 일부입니다.
# 모델 가져오기
@st.cache_resource
def load_model():
    model = genai.GenerativeModel('gemini-1.5-flash')
print("model loaded...")
    return model

model = load_model()

In [ ]:
# 세션별 이력 관리
## 별도의 딕셔너리 데이터 구조로 관리하지 않고, 제미나이 api의 chatsession 그대로 사용
if "chat_session" not in st.session_state:
    st.session_state["chat_session"] = model.start_chat(history=[]) # ChatSession 반환

In [ ]:
# 메시지 출력
if prompt := st.chat_input("메시지를 입력하세요."):
    with st.chat_message("user"):
        st.markdown(prompt)
    with st.chat_message("ai"):
        response = st.session_state.chat_session.send_message(prompt)        
        st.markdown(response.text)

In [ ]:
#대화이력 출력
for content in st.session_state.chat_session.history:
    with st.chat_message("ai" if content.role == "model" else "user"):
        st.markdown(content.parts[0].text)

- ChateSession 객체의 history에는 사용자의 언어모델이 나눈 대화이력이 role과 parts로 구분되어 들어 있음.
- 세션 객체 들어 있는 history 정보를 활용하면 사용자와 제미나이가 나누었던 대화가 화면에 이력으로 출력됨.
- history에 있는 role을 스트림릿에 등록되어 있는 아바타 이름과 맞추기 위해 chat_message 메서드 호출시 'model'을 'ai'로 변경하는 코드 추가